## Teste de aderência

Utilizado em amostras com variáveis dicotômicas, testa se a proporção de
sucesso observada na amostra (𝑝̂) pertence a uma população com um
determinado valor de p. 

### Teste Binomial

Em uma amostra de tamanho 20 foram observados 5 sucessos. Testar se p>0.2
com nível de significância de 5%.

In [47]:
# Importação da função de teste binomial do SciPy
from scipy.stats import binomtest

# Função para realizar um teste binomial com saída formatada
def TesteBinomial(k = int, n = int, p = float, alternativa = 'two-sided', conf = 0.95):
    # Realiza o teste binomial com os parâmetros fornecidos
    teste = binomtest(k = k, n = n, p = p, alternative = alternativa)

    # Obtém o intervalo de confiança da proporção
    ic = teste.proportion_ci(conf)

    # Define se a hipótese nula é aceita ou rejeitada com base no p-valor
    h0 = 'ACEITA' if teste.pvalue > 1 - conf else 'REJEITADA'

    # Define a hipótese alternativa com base no argumento 'alternativa'
    h1 = {
        'two-sided': f'A verdadeira proporção é diferente de {p}',
        "greater": f"A verdadeira proporção é maior que {p}",
        "less": f"A verdadeira proporção é menor que {p}"
    }

    # Mensagem de saída com resultados e conclusões do teste
    mensagem_resultado = f"""
    Número de sucessos: {k}, Número de tentativas: {n}, p-valor: {teste.pvalue}
    
    intervalos de confiança {int(100*conf)}%: {ic}
    
    H0: p = {p}
    H1: {h1[alternativa]}
    
    Resultado:
    H0: {h0}
    """
    return mensagem_resultado

In [48]:
# Testando teste binomial
print(TesteBinomial(5,20,0.2,'greater'))


    Número de sucessos: 5, Número de tentativas: 20, p-valor: 0.3703517360973313
    
    intervalos de confiança 95%: ConfidenceInterval(low=0.1040808359101361, high=1.0)
    
    H0: p = 0.2
    H1: A verdadeira proporção é maior que 0.2
    
    Resultado:
    H0: ACEITA
    


### Teste Qui-quadrado

Utilizado para testar se a frequência observada na amostra difere
significativamente da frequência esperada especificada por uma distribuição de
probabilidade.

In [49]:
# Importações necessárias
from scipy.stats import binom, chisquare, poisson # Distribuição binomial, Poisson e teste do Qui-Quadrado
import numpy as np

In [50]:
# Função para calcular valores esperados de uma distribuição Binomial
def binomial_valores_esperados(p, n, total):
    # Calcula as probabilidades de uma distribuição binomial e multiplica pelo total
    # 'binom.pmf(i, n, p)' calcula a probabilidade binomial para o valor 'i' dado 'n' tentativas e probabilidade 'p'
    valores = [binom.pmf(i, n, p) * total for i in range(n + 1)]
    
    return valores  # Retorna a lista com os valores esperados

In [51]:
# Função para calcular valores esperados da distribuição Poisson
def poisson_exp(p, n):
    # Cria uma lista vazia para armazenar os valores calculados
    valores = []
    
    # Calcula a probabilidade de Poisson para cada valor de 0 até n
    for i in range(0, n + 1):
        # 'poisson.pmf(i, p)' calcula a probabilidade de Poisson para o valor 'i' com parâmetro 'p'
        valores.append(poisson.pmf(i, p))
    
    # Adiciona o complemento das probabilidades (1 - soma das probabilidades calculadas)
    # Isso é útil para representar a probabilidade de valores maiores que 'n'.
    valores.append(1 - sum(valores))
    
    return valores  # Retorna a lista com os valores esperados


In [52]:
# Função de teste de aderência do Qui-Quadrado
def ChiTeste(f_observada, f_esperada):
    # Aplica o teste de aderência Qui-Quadrado usando as funções do SciPy
    # 'f_obs' são as frequências observadas e 'f_exp' as frequências esperadas.
    teste = chisquare(f_obs=f_observada, f_exp=f_esperada)
    
    # Formatação da mensagem de saída, com os resultados do teste
    mensagem = f"""
    Estatística do teste: {teste.statistic}  # Estatística do teste Qui-Quadrado.
    p-valor: {teste.pvalue}  # p-valor do teste, indicando a probabilidade do resultado ser aleatório.
    """
    
    return mensagem  # Retorna a mensagem com os resultados

Abaixo, temos o número observado de falhas mecânicas, por hora, em uma linha
de montagem, a partir de um experimento com duração de 40 horas. Um
engenheiro afirma que o processo descrito, seguem uma distribuição de Poisson
com média igual a 3.2.Testar com α=0.05.

### Teste de kolmogorov-Smirnov

Avalia se os dados amostrais se aproximam razoavelmente de uma
determinada distribuição

cenário:

Você é um analista de dados de uma empresa que coleta informações sobre a quantidade de horas que seus funcionários trabalham por semana. Eles têm uma política de horário de trabalho, mas querem saber se a distribuição das horas trabalhadas segue um padrão específico.

A empresa afirma que, com base em estudos anteriores, a distribuição das horas semanais de trabalho dos funcionários deveria seguir uma distribuição normal, com média de 40 horas e desvio padrão de 5 horas. No entanto, você suspeita que a distribuição real pode ser diferente, talvez devido a mudanças nas políticas de trabalho remoto ou outros fatores inesperados.

Objetivo:

Você deve realizar o teste de Kolmogorov-Smirnov para verificar se a distribuição das horas semanais de trabalho segue uma distribuição normal com média de 40 horas e desvio padrão de 5 horas. Para isso, você tem uma amostra de dados de 500 funcionários sobre suas horas trabalhadas nas últimas semanas.

Hipótestes:

- Hipótese nula (H0): As horas semanais seguem uma distribuição normal com média de 40 horas e desvio padrão de 5 horas.

- Hipótese alternativa (H1): As horas semanais não seguem uma distribuição normal com média de 40 horas e desvio padrão de 5 horas.

In [53]:
# importações
import numpy as np

In [54]:
# simulando dados
dados = np.random.normal(40,5,50)

In [55]:
# Importando teste
from scipy.stats import kstest
# Importa o teste de Kolmogorov-Smirnov da biblioteca SciPy.

# Construindo função do teste
def KomogorovTest(amostra, densidade, N=20, alternativa='two-sided', metodo='auto', conf=0.95):
    # Realiza o teste de Kolmogorov-Smirnov para comparar a amostra com uma distribuição teórica.
    # Parâmetros:
    # - amostra: os dados empíricos a serem testados.
    # - densidade: a cdf (função de distribuição acumulada) da distribuição teórica (ex: 'norm').
    # - N: número de pontos usados na cdf se ela for uma string (default = 20).
    # - alternativa: 'two-sided', 'less' ou 'greater'.
    # - metodo: método de cálculo do p-valor ('auto', 'exact', 'asymp').
    # - conf: nível de confiança para decisão da hipótese nula.

    stats, pvalue = kstest(rvs=amostra, cdf=densidade, N=N, alternative=alternativa, method=metodo)
    # Aplica o teste KS e obtém a estatística e o p-valor.

    print(pvalue)
    # Imprime o p-valor (pode ser removido se quiser retorno limpo).

    saida = f"""
    =========== Informações do Teste ============
    
    - Estatística do Teste: {stats}
    
    - P-Valor: {pvalue}
    
    - H0: {True if pvalue > 1 - conf else False}
    
    """
    # Formata os resultados e avalia se a hipótese nula é aceita (com base em p > 1 - conf).

    return saida

In [56]:
# Testando Hipóteses:
from scipy.stats import norm

print(KomogorovTest(dados,norm.cdf))

0.0

    =========== Informações do Teste ============
    
    - Estatística do Teste: 1.0
    
    - P-Valor: 0.0
    
    - H0: False
    
    


Rejeitamos H0, concluindo que as distribuições são diferentes. Em um nível de confiança de 0,05%

### Teste de Lilliefors

Esse teste é utilizado para verificar a normalidade dos dados.

Cenário:

Uma equipe de RH está analisando o número de dias de afastamento médico dos funcionários de uma empresa ao longo de um ano. Eles suspeitam que esse número de dias por funcionário segue uma distribuição normal, mas não conhecem a média nem o desvio padrão populacional. A equipe deseja verificar se essa suposição de normalidade é razoável com base nos dados coletados de 60 funcionários selecionados aleatoriamente.

Hipótetse:
- H0: Os dados sobre os funcionários seguem uma distribuição Normal

- H1: Os dados sobre os funcionários não seguem uma distribuição Normal

In [57]:
# Simulando amostra
dados = norm.rvs(0,1,60)
dados

array([ 0.45052859, -1.49896787, -0.14074084, -1.670475  , -0.78967155,
       -1.06548453,  0.14041997, -0.96775735,  0.00383305, -0.39746301,
       -0.64592981, -0.03946952,  0.15504041, -0.67513617, -0.40684639,
       -0.00807874, -0.01445527, -0.64666024, -2.27731149,  1.00174953,
       -1.77063587,  1.61971343,  0.07166404, -1.89706832,  1.02851702,
       -2.0857723 , -0.34463575, -0.61110228,  0.18750571,  0.61063083,
        1.19408213, -2.42343393,  1.81210423, -0.56325557, -1.22597969,
       -0.55407092,  0.54721181,  1.52962456, -1.0770806 ,  0.32567823,
        0.95179051,  1.08775471, -0.32604493,  1.63592488,  1.35061357,
        0.13072375, -0.86264936,  0.85512686,  1.08679263,  1.01050923,
        0.12332546,  1.20857028, -1.13511577,  0.19423546,  1.17630679,
        0.61868007,  0.16980616, -0.02799717, -0.10938564,  1.90450928])

In [58]:
# Importação do teste
from statsmodels.stats.diagnostic import lilliefors
# Importa o teste de Lilliefors, usado para verificar aderência a uma distribuição (ex: normal).

# Criando função do teste
def Teste_lilliefors(dados, distribuicao="norm", pvalmethod='table', conf=0.95):
    # Função que realiza o teste de Lilliefors em um conjunto de dados.
    # Parâmetros:
    # - dados: vetor com os dados a serem testados.
    # - distribuicao: distribuição alvo (default = "norm" para normal).
    # - pvalmethod: método para p-valor ('table' = usa tabela, 'approx' = aproximação).
    # - conf: nível de confiança (default = 95%).

    stats, p = lilliefors(dados, distribuicao, pvalmethod)
    # Aplica o teste e retorna estatística do teste e p-valor.

    saida = f"""
    ============ Informações sobre o Teste ===========
    
    - Estatística do Teste:{stats}
    
    - P valor: {p}
    
    - H0: {True if p > 1 - conf else False}
    """
    # Formata uma string com os resultados e se a hipótese nula é aceita ou não.

    return saida

In [59]:
# Testando Hipótestes
print(Teste_lilliefors(dados))


    ============ Informações sobre o Teste ===========
    
    - Estatística do Teste:0.07331358887797457
    
    - P valor: 0.6261865331676557
    
    - H0: True
    
